# Fuzzy String Fast Matching TF-IDF

Use TF-IDF method to match similar company names

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import time
import re
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct


pd.set_option('display.max_colwidth', -1)
names2 =  pd.read_csv('bloomberg companies.csv')
names =  pd.read_csv('G.csv', delimiter='|')
names.columns = ['Key', 'Name']
print('The shape: %d x %d' % names.shape)
print(names.head())
names2.head()

The shape: 450256 x 2
      Key                        Name
0  634022  PRIMCOM SA                
1  324497  The David Isaacs Fund     
2  280848  Bramor Enterprises Limited
3  432662  NAVEXIM S.A.              
4  524224  Magal Group SA            


,Number,Name,Key
0,1,!J INC,1438823
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
3,4,#1 PAINTBALL CORP,1433777
4,5,$ LLC,1427189


In [7]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "McDonalds":')
ngrams('McDonalds')

All 3-grams in "McDonalds":


['McD', 'cDo', 'Don', 'ona', 'nal', 'ald', 'lds']

In [8]:
company_names = names['Name']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [9]:
print(tf_idf_matrix[0])
ngrams('!J INC')

  (0, 33432)	0.346336976239
  (0, 34978)	0.377398243742
  (0, 23135)	0.408408481581
  (0, 28340)	0.406574410515
  (0, 14471)	0.294909925079
  (0, 31634)	0.388846153349
  (0, 28063)	0.340799164551
  (0, 1957)	0.226282478229


['!J ', 'J I', ' IN', 'INC']

In [10]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

In [9]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        if sparserows[index] != sparsecols[index] :
            left_side[index] = name_vector[sparserows[index]]
            right_side[index] = name_vector[sparsecols[index]]
            similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [10]:
matches_df = get_matches_df(matches, company_names, top=0)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)
matches_df[matches_df['left_side'] == 'HAMPSTEAD ASSOCIATES LLC']

,left_side,right_side,similairity
553568,HAMPSTEAD ASSOCIATES LLC,HAMPSTEAD ASSOCIATES INC,0.961686


In [11]:
matches_df.sort_values(['similairity'], ascending=False).head(10)

,left_side,right_side,similairity
1298576,VILLERE ST DENIS J & CO /ADV,VILLERE ST DENIS J & CO /ADV,0.999985
1298586,VILLERE ST DENIS J & CO /ADV,VILLERE ST DENIS J & CO /ADV,0.999985
968490,PG INVESTORS II INC /ADV,PG INVESTORS III INC /ADV,0.999648
968500,PG INVESTORS III INC /ADV,PG INVESTORS II INC /ADV,0.999648
968510,PG INVESTORS INC /ADV,PG INVESTORS II INC /ADV,0.998993
968491,PG INVESTORS II INC /ADV,PG INVESTORS INC /ADV,0.998993
297517,CSK CORP /FI,SK CORP /FI,0.998968
1121593,SK CORP /FI,CSK CORP /FI,0.998968
54674,ALLEN & CO INC /BD,MULLEN & CO INC /BD,0.998930
846418,MULLEN & CO INC /BD,ALLEN & CO INC /BD,0.998930


In [12]:
print(names.sample())
matches_df[matches_df['left_side'] == 'DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168']

       Number                Name      Key
87901  87902   BROWNLIE WILLIAM R  1346227


,left_side,right_side,similairity
330312,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 16,0.968790
330313,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 1,0.949965
330314,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 161,0.942374
330315,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 160,0.941028
330316,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 163,0.940108
330317,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 169,0.938703
330318,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 167,0.938629
330319,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 19,0.937084
330320,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 168,DEFINED ASSET FUNDS MUNICIPAL INVT TR FD MON PYMT SER 10,0.934729
